# 스파크 스트리밍 실습 
> 파일을 읽어서 파이썬 스트리밍 실습을 합니다

## 목표
1. 특정 경로에 존재하는 Json 파일을 스트리밍으로 읽는 실습을 수행합니다
1. 스트리밍 집계 함수를 실습합니다
1. 스트리밍 변환 함수를 실습합니다
1. 여러 스트림을 조인하는 연산을 실습합니다
1. 이벤트 시간 기준의 텀블링 윈도우 처리 실습
1. 워터마크를 이용하여 1분 단위로 접속한 이용자의 수를 가지는 테이블을 구성하고 조회합니다
1. UDF 를 이용하여, 워터마크와 윈도우 함수를 이용하여 누적 최고 매출 이용자를 가지는 테이블을 구성하고 조회합니다
1. DataFrame 을 이용하여, 동일한 기능을 실습합니다

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .appName("foo-bar") \
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)
# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

### 스파크 스트리밍 처리에서 자주 활용되는 함수들

In [2]:
from random import uniform
from time import sleep
from IPython.display import display, clear_output
# 스트림 테이블을 주기적으로 조회하는 함수
def displayStream(name, sql, iterations, sleep_secs):
    i = 1
    for x in range(iterations):
        clear_output(wait=True)
        display('[' + name + '] Iteration: '+str(i)+', Score: '+str(uniform(0, 1)))
        spark.sql(sql).show(truncate=False)
        sleep(sleep_secs)    
        i += 1

In [3]:
from IPython.display import JSON
# 해당 스트리밍 쿼리의 마지막 상태를 확인하는 함수
def checkLastProgress(query):
    JSON(query.lastProgress)

In [4]:
# 해당 스트리밍 쿼리를 재실행 시에, 이미 시작된 쿼리를 사전에 종료하기 위한 함수
def stopQueryIfStarted(query):
    try:
        query.stop()
    except:
        pass

In [5]:
# 테이블 목록 출력
def showTables():
    spark.sql("show tables").show()

## 1. 특정 경로에 존재하는 Json 파일을 읽어서 스트리밍 파이프라인을 구성합니다

In [6]:
static = spark.read.json("data/activity-data")
dataSchema = static.schema
static.printSchema()
static.show(2, truncate=False)

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+-----------+
|Arrival_Time |Creation_Time      |Device  |Index|Model |User|gt   |x           |y           |z          |
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+-----------+
|1424686734992|1424688581040070924|nexus4_2|5    |nexus4|g   |stand|-3.814697E-4|0.025878906 |0.023727417|
|1424686735190|1424688581245179566|nexus4_2|46   |nexus4|g   |stand|-0.008926392|-0.047821045|0.011978149|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+-

In [7]:
streaming = spark.readStream.schema(dataSchema) \
    .option("maxFilesPerTrigger", 1).json("data/activity-data")
spark.conf.set("spark.sql.shuffle.partitions", 5)  # 로컬 모드에서는 너무 많은 파티션은 오히려 성능을 떨어뜨리므로 셔플 수를 줄입니다

## 2. 스트리밍 집계 함수를 실습합니다

In [8]:
activityCounts = streaming.groupBy("gt").count()  # 스트리밍은 바로 출력 대신에 다른 스트리밍 싱크에 보내기 위한 로직만 먼저 작성합니다

# 아래와 같이 "activity_counts" 라는 임의의 "memory" 테이블에 매번 모든 결과를 "complete" 하게 전송하는 activityQuery 를 생성합니다
activityQuery = activityCounts.writeStream \
    .queryName("activity_counts") \
    .format("memory") \
    .outputMode("complete") \
    .start()

# 상태를 확인하기 위해서 항상 스트리밍 쿼리를 별도의 객체로 만들어 두는 것이 편리합니다.
activityQuery.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [9]:
# 위와 같이 start 한 이후, 백그라운드에서 코드를 수행시키기 위해서 아래와 같이 실행합니다
DEBUG = False
if DEBUG:
    activityQuery.id  # get the unique identifier of the running query that persists across restarts from checkpoint data

    activityQuery.runId       # get the unique id of this run of the query, which will be generated at every start/restart

    activityQuery.name        # get the name of the auto-generated or user-specified name

    activityQuery.explain()   # print detailed explanations of the query

    activityQuery.stop()      # stop the query

    activityQuery.awaitTermination()   # block until query is terminated, with stop() or with error

    activityQuery.exception       # the exception if the query has been terminated with error

    activityQuery.recentProgress  # an array of the most recent progress updates for this query

    activityQuery.lastProgress    # the most recent progress update of this streaming query


#### 현재 활성화된 스트리밍 쿼리에 대한 확인 및 조회

In [10]:
spark.streams.active  # get the list of currently active streaming queries

# spark.streams.get(id)  # get a query object by its unique id

# 라이브 어플리케이션의 경우는 백그라운드에서 항상 동작해야 하므로, 아래와 같이 어플리케이션이 종료될 때까지 대기하는 명령을 수행해야 합니다.
# spark.streams.awaitAnyTermination()  # block until any one of them terminates

In [11]:
displayStream("activity_counts", "select * from activity_counts", 3, 1)

'[activity_counts] Iteration: 3, Score: 0.4339247963572942'

+----------+-----+
|gt        |count|
+----------+-----+
|sit       |12309|
|stand     |11384|
|stairsdown|9365 |
|walk      |13256|
|stairsup  |10452|
|null      |10449|
|bike      |10796|
+----------+-----+



In [12]:
displayStream("Counts of Activities", "select * from activity_counts", 3, 1)

'[Counts of Activities] Iteration: 3, Score: 0.32484175874831367'

+----------+-----+
|gt        |count|
+----------+-----+
|sit       |73855|
|stand     |68309|
|stairsdown|56192|
|walk      |79536|
|stairsup  |62710|
|null      |62688|
|bike      |64781|
+----------+-----+



## 3. 스트리밍 변환 함수를 실습합니다

In [13]:
simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'")) \
    .where("stairs") \
    .where("gt is not null") \
    .select("gt", "model", "arrival_time", "creation_time") \
    .writeStream \
    .queryName("simple_transform") \
    .format("memory") \
    .outputMode("append") \
    .start()

In [14]:
displayStream("Counts of Activities", "select * from simple_transform", 3, 1)

'[Counts of Activities] Iteration: 3, Score: 0.5816592986962856'

+--------+------+-------------+-------------------+
|gt      |model |arrival_time |creation_time      |
+--------+------+-------------+-------------------+
|stairsup|nexus4|1424687983719|1424687981726802718|
|stairsup|nexus4|1424687984000|1424687982009853255|
|stairsup|nexus4|1424687984404|1424687982411977009|
|stairsup|nexus4|1424687984805|1424687982814351277|
|stairsup|nexus4|1424687985210|1424687983217500861|
|stairsup|nexus4|1424687985620|1424687983620332892|
|stairsup|nexus4|1424687986016|1424687984023164923|
|stairsup|nexus4|1424687986420|1424687984425874884|
|stairsup|nexus4|1424687986820|1424687984828822915|
|stairsup|nexus4|1424687987225|1424687985231654946|
|stairsup|nexus4|1424687987625|1424687985634469017|
|stairsup|nexus4|1424687987992|1424687986002114280|
|stairsup|nexus4|1424687988191|1424689834237427627|
|stairsup|nexus4|1424687988392|1424689834438660537|
|stairsup|nexus4|1424687988592|1424689834640076553|
|stairsup|nexus4|1424687988794|1424689834841675674|
|stairsup|ne

In [15]:
checkLastProgress(activityQuery)

In [16]:
showTables()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|        | activity_counts|       true|
|        |simple_transform|       true|
+--------+----------------+-----------+



## 4. 여러 스트림을 조인하는 연산을 실습합니다

In [17]:
# 아래와 같이 데이터소스를 처음에 읽어서 해당 데이터프레임을 만들어두면, 해당 스트림을 사용하면 파일을 처음부터 읽어서 테스트가 용이합니다
deviceModelStats = None
stopQueryIfStarted(deviceModelStats)
historicalAgg = static.groupBy("gt", "model").avg()
deviceModelStats = streaming.drop("Arrival_Time", "Creation_Time", "Index") \
    .cube("gt", "model").avg() \
    .join(historicalAgg, ["gt", "model"]) \
    .writeStream \
    .queryName("device_counts") \
    .format("memory") \
    .outputMode("complete") \
    .start()

In [18]:
displayStream("device_counts", "select * from device_counts", 5, 1)

'[device_counts] Iteration: 5, Score: 0.6048293125016144'

+----------+------+----------------------+---------------------+----------------------+---------------------+----------------------+------------------+----------------------+---------------------+----------------------+
|gt        |model |avg(x)                |avg(y)               |avg(z)                |avg(Arrival_Time)    |avg(Creation_Time)    |avg(Index)        |avg(x)                |avg(y)               |avg(z)                |
+----------+------+----------------------+---------------------+----------------------+---------------------+----------------------+------------------+----------------------+---------------------+----------------------+
|bike      |nexus4|0.02351254447093369   |-0.01304747996973879 |-0.08360475809007031  |1.4247511339065625E12|1.42475212569564083E18|326469.69726045156|0.02333181232446801   |-0.009512011859706009|-0.08247843125455155  |
|null      |nexus4|-0.0030250122150636423|-0.004107545014106613|0.005961452067049493  |1.4247490083833638E12|1.424749925

## 5. 이벤트 시간 기준의 텀블링 윈도우 처리 실습

In [19]:
withEventTime = streaming.selectExpr("*", "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

In [20]:
eventPerWindow = None
stopQueryIfStarted(eventPerWindow)
eventPerWindow = withEventTime.groupBy(window(col("event_time"), "10 minutes")).count() \
     .writeStream \
     .queryName("pyevents_per_window") \
     .format("memory") \
     .outputMode("complete") \
     .start()

In [21]:
displayStream("pyevents_per_window", "select * from pyevents_per_window", 5, 3)

'[pyevents_per_window] Iteration: 5, Score: 0.08150279462775711'

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2015-02-24 20:50:00, 2015-02-24 21:00:00]|13159|
|[2015-02-24 22:00:00, 2015-02-24 22:10:00]|11752|
|[2015-02-23 21:30:00, 2015-02-23 21:40:00]|8799 |
|[2015-02-23 19:20:00, 2015-02-23 19:30:00]|8660 |
|[2015-02-24 21:30:00, 2015-02-24 21:40:00]|10975|
|[2015-02-24 22:10:00, 2015-02-24 22:20:00]|9138 |
|[2015-02-23 19:30:00, 2015-02-23 19:40:00]|8805 |
|[2015-02-23 19:40:00, 2015-02-23 19:50:00]|7685 |
|[2015-02-23 22:20:00, 2015-02-23 22:30:00]|9285 |
|[2015-02-22 09:40:00, 2015-02-22 09:50:00]|3    |
|[2015-02-24 20:20:00, 2015-02-24 20:30:00]|9914 |
|[2015-02-24 21:20:00, 2015-02-24 21:30:00]|11632|
|[2015-02-24 23:00:00, 2015-02-24 23:10:00]|13126|
|[2015-02-24 23:10:00, 2015-02-24 23:20:00]|14686|
|[2015-02-24 22:40:00, 2015-02-24 22:50:00]|11634|
|[2015-02-24 22:50:00, 2015-02-24 23:00:00]|8458 |
|[2015-02-23 23:30:00, 2015-02-

In [22]:
eventPerUserWindow = None
stopQueryIfStarted(eventPerUserWindow)
eventPerUserWindow = withEventTime.groupBy(window(col("event_time"), "10 minutes"), "User").count() \
     .writeStream \
     .queryName("pyevents_per_user_window") \
     .format("memory") \
     .outputMode("complete") \
     .start()

In [23]:
displayStream("pyevents_per_user_window", "select * from pyevents_per_user_window order by count desc limit 5", 5, 1)

'[pyevents_per_user_window] Iteration: 5, Score: 0.562776291874881'

+------------------------------------------+----+-----+
|window                                    |User|count|
+------------------------------------------+----+-----+
|[2015-02-23 19:50:00, 2015-02-23 20:00:00]|g   |8030 |
|[2015-02-24 23:40:00, 2015-02-24 23:50:00]|e   |7484 |
|[2015-02-24 23:00:00, 2015-02-24 23:10:00]|b   |7469 |
|[2015-02-24 20:40:00, 2015-02-24 20:50:00]|i   |7299 |
|[2015-02-24 21:20:00, 2015-02-24 21:30:00]|f   |6688 |
+------------------------------------------+----+-----+



In [24]:
static.show(10, truncate=False)

+-------------+-------------------+--------+-----+------+----+-----+-------------+-------------+------------+
|Arrival_Time |Creation_Time      |Device  |Index|Model |User|gt   |x            |y            |z           |
+-------------+-------------------+--------+-----+------+----+-----+-------------+-------------+------------+
|1424686734992|1424688581040070924|nexus4_2|5    |nexus4|g   |stand|-3.814697E-4 |0.025878906  |0.023727417 |
|1424686735190|1424688581245179566|nexus4_2|46   |nexus4|g   |stand|-0.008926392 |-0.047821045 |0.011978149 |
|1424686735395|1424686733397706064|nexus4_1|79   |nexus4|g   |stand|3.356934E-4  |0.02507019   |-0.005996704|
|1424686735593|1424688581647920045|nexus4_2|126  |nexus4|g   |stand|0.0038909912 |-0.0093688965|0.0023651123|
|1424686735795|1424688581849427613|nexus4_2|166  |nexus4|g   |stand|0.0038909912 |-0.0050964355|0.01838684  |
|1424686735998|1424686734002381357|nexus4_1|199  |nexus4|g   |stand|-0.0028686523|-5.645752E-4 |0.028182983 |
|142468673

In [25]:
static.groupBy("Device", "User").count().sort(desc("count")).limit(5).show()

+--------+----+-----+
|  Device|User|count|
+--------+----+-----+
|nexus4_2|   e|49605|
|nexus4_2|   i|46887|
|nexus4_1|   f|46745|
|nexus4_2|   g|46538|
|nexus4_1|   e|46425|
+--------+----+-----+



In [26]:
userRank = None
stopQueryIfStarted(userRank)

# 해당 그룹 파티션 내에서 X, Y, Z 역순 정렬 후에, 유일한 번호를 매겨서 출력
userRank = withEventTime \
    .withWatermark("event_time", "10 minutes") \
    .groupBy("User", window(col("event_time"), "10 minutes")).count() \
    .writeStream \
    .queryName("user_rank") \
    .format("memory") \
    .outputMode("complete") \
    .start()

In [27]:
displayStream("user_rank", "select * from user_rank order by window desc, User, count desc", 1, 1)

'[user_rank] Iteration: 1, Score: 0.6162055657482368'

+----+------+-----+
|User|window|count|
+----+------+-----+
+----+------+-----+



## 5. 워터마크를 이용하여 1분 단위로 접속한 이용자의 수를 가지는 테이블을 구성하고 조회합니다

In [28]:
df = spark.read.option("header", "true").csv("data/tbl_user.csv")
df.printSchema()
df.show()

root
 |-- u_id: string (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: string (nullable = true)

+----+----------+--------+--------+
|u_id|    u_name|u_gender|u_signup|
+----+----------+--------+--------+
|   1|    정휘센|      남|19580808|
|   2|  김싸이언|      남|19590201|
|   3|    박트롬|      여|19951030|
|   4|    청소기|      남|19770329|
|   5|유코드제로|      여|20021029|
|   6|  윤디오스|      남|20040101|
|   7|  임모바일|      남|20040807|
|   8|  조노트북|      여|20161201|
|   9|  최컴퓨터|      남|20201124|
+----+----------+--------+--------+



In [29]:
df.rdd.zipWithIndex().collect()

[(Row(u_id='1', u_name='정휘센', u_gender='남', u_signup='19580808'), 0),
 (Row(u_id='2', u_name='김싸이언', u_gender='남', u_signup='19590201'), 1),
 (Row(u_id='3', u_name='박트롬', u_gender='여', u_signup='19951030'), 2),
 (Row(u_id='4', u_name='청소기', u_gender='남', u_signup='19770329'), 3),
 (Row(u_id='5', u_name='유코드제로', u_gender='여', u_signup='20021029'), 4),
 (Row(u_id='6', u_name='윤디오스', u_gender='남', u_signup='20040101'), 5),
 (Row(u_id='7', u_name='임모바일', u_gender='남', u_signup='20040807'), 6),
 (Row(u_id='8', u_name='조노트북', u_gender='여', u_signup='20161201'), 7),
 (Row(u_id='9', u_name='최컴퓨터', u_gender='남', u_signup='20201124'), 8)]

In [30]:
df.withColumn("id", monotonically_increasing_id() + 1).show()

+----+----------+--------+--------+---+
|u_id|    u_name|u_gender|u_signup| id|
+----+----------+--------+--------+---+
|   1|    정휘센|      남|19580808|  1|
|   2|  김싸이언|      남|19590201|  2|
|   3|    박트롬|      여|19951030|  3|
|   4|    청소기|      남|19770329|  4|
|   5|유코드제로|      여|20021029|  5|
|   6|  윤디오스|      남|20040101|  6|
|   7|  임모바일|      남|20040807|  7|
|   8|  조노트북|      여|20161201|  8|
|   9|  최컴퓨터|      남|20201124|  9|
+----+----------+--------+--------+---+



In [31]:
df = spark.sql("select 'a,b' as col1")
df2 = df.withColumn("codes" , explode(split("col1" , ","))).drop("col1")
df2.show()

+-----+
|codes|
+-----+
|    a|
|    b|
+-----+



In [32]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [33]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|[James, , Smith]    |36636|M     |3100  |
|[Michael, Rose, ]   |40288|M     |4300  |
|[Robert, , Williams]|42114|M     |1400  |
|[Maria, Anne, Jones]|39192|F     |5500  |
|[Jen, Mary, Brown]  |     |F     |-1    |
+--------------------+-----+------+------+



In [34]:
df2.selectExpr("name.firstname as first", "name.middlename", "name.lastname").show()

+-------+----------+--------+
|  first|middlename|lastname|
+-------+----------+--------+
|  James|          |   Smith|
|Michael|      Rose|        |
| Robert|          |Williams|
|  Maria|      Anne|   Jones|
|    Jen|      Mary|   Brown|
+-------+----------+--------+

